In [1]:
import numpy as np
import pandas as pd
import random
import sys

import keras
import tensorflow

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, RNN
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv('scraped_lyrics.csv')

In [3]:
df.head()

,artist,title,song_id,lyrics
0,Billie Eilish,COPYCAT,Billie-eilish-copycat,"Don't be cautious, don't be kind, You committ..."
1,Billie Eilish,​idontwannabeyouanymore,Billie-eilish-idontwannabeyouanymore,"Don't be that way, 'Fall apart twice a day', ..."
2,Billie Eilish,​my boy,Billie-eilish-my-boy,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b..."
3,Billie Eilish,​watch,Billie-eilish-watch,"'Lips meet teeth and tongue', 'My heart skips..."
4,Billie Eilish,​​party favor,Billie-eilish-party-favor,"'Hey, leave a message', 'Hey, call me back ..."


In [11]:
# with open('only_lyrics.txt', 'w') as f:
#     f.write(str(lyrics))

In [2]:
import glob
glob.glob('only_lyrics.txt')

['only_lyrics.txt']

In [4]:
text = (open('only_lyrics.txt').read().lower())
print(f'Corpus length: {len(text)}')

Corpus length: 140756


In [6]:
%%time
# Vectorizing sequences of characters

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print(f'Number of sequences: {len(sentences)}')

chars = sorted(list(set(text)))
print(f'Unique chracters: {len(chars)}')
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 46899
Unique chracters: 55
Vectorization...
CPU times: user 707 ms, sys: 48.1 ms, total: 755 ms
Wall time: 754 ms


In [16]:
# print(text)

In [44]:
# Building up LSTM model

model = keras.Sequential()
model.add(LSTM(400, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 60, 400)           729600    
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 400)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 60, 400)           1281600   
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 400)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 400)               1281600   
_________________________________________________________________
dropout_7 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 55)                22055     
Total para

In [ ]:
%%time
"""
DON'T RUN THIS LOCALLY!!!!
"""

model.fit(x,y, epochs=35, batch_size=32)
model.save('lyric_generator_2.1.20_3layer_400_neurons.h5')

Epoch 1/35
 3872/46899 [=>............................] - ETA: 28:48 - loss: 3.9578

In [31]:
def reweight_distribution(original_distribution, temperature=0.6):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)

In [32]:
# Reweighting probability distribution to introduce extra randomness into text generation
# Prior iterations performed well with a 'temperature' ~0.6 to balance creativity and coherence

def sample(preds, temperature=0.6):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
# Testing loaded model

from keras.models import load_model
model = load_model('lyric_generator_small_test_2.h5')

In [13]:
import glob
glob.glob('lyric_generator_small_test.h5')

['lyric_generator_small_test.h5']

In [43]:
# Generatic sample lyrics to check model efficacy

temperature = 0.6

for song in range(1, 60):
    print(f'\n\nGrammy Award Winning Song #{song}\n')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('-------- Generating with seed lyrics:  "' + generated_text + '"\n')
    
    # Number of characters assuming an average song length of 200-300 words
    for i in range(800):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1
            
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)



Grammy Award Winning Song #1

-------- Generating with seed lyrics"ant me to be yours? well, then you gotta be mine', 'and if y"

ou theud tot toout soud , 'ak, 'toue toon' to the tat totr tof too tos the ck the toon tout ther you thou ou tou ore soal fouc,  oo ing aow the pou toth thy bof tot thann art tot s out worot cou th the tot waft it sot to bu he tay in tom thou tou sou an way a tey the thang son tove the the sout sou the roon whe tut slit to ther i nt to 'ron soy a tood to's 'ou the  ou', 'to thou thert 'ctou the', 'then withe thout tou mus the lofe it ing a thot whe too sot the ta dabd tou to the tou', 'ou to tar the thand iot the the to ton tot toos i cou gout boby sotr gok tou', 'ou coutt thi gou', 'lo toog the ton tot thou wuth  li ta tour et thet an tit , 'the 'a the sou sout ong to too thath you toor th, 'the cout rou', 'ou tol the,  ouy mytu so tou towit the tow tout to to the yout tho tonr tou t', 'o

Grammy Award Winning Song #2

-------- Generating with seed lyrics

KeyboardInterrupt: 